In [1]:
import schedule
import time
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib
import pypyodbc as podbc


In [2]:
# sql server cnx setup
cnx = podbc.connect("Driver={SQL Server Native Client 11.0};"
                                 "Server=10.0.0.1,1433;"
                                 "Database=Covid19_global;"
                                 "Trusted_Connection=yes;")

quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                                 "Server=myaddress;"
                                 "Database=Covid19_global;"
                                 "Trusted_Connection=yes;")

engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))



def data_ETL():
    # Data collection
    data_confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    data_deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    data_recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
    data_confirmed_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
    data_deaths_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
    
    
    # cration Dates list
    column_dates = data_confirmed_US.columns
    column_dates = column_dates.drop(labels =['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
                                          'Country_Region', 'Lat', 'Long_', 'Combined_Key'])

    # supprimer les colone inutile dans US confirmed et deaths df
    data_confirmed_US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'], axis = 1, inplace=True)
    data_deaths_US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'], axis = 1, inplace=True)
    
    # calcul de la moyenne lat et log
    lat_long_avg = data_confirmed_US.groupby(['Province_State', 'Country_Region'], as_index=False).agg({'Lat':'median', 'Long_':'median'})

    # somme des valeurs 
    data_confirmed_US = data_confirmed_US.groupby(['Province_State', 'Country_Region'], as_index = False)[column_dates].sum()
    data_deaths_US = data_deaths_US.groupby(['Province_State', 'Country_Region'], as_index = False)[column_dates].sum()
    
    # Left join avg lat et long
    data_confirmed_US = data_confirmed_US.merge(lat_long_avg, how = 'left',
                                               left_on = ['Province_State', 'Country_Region'],
                                               right_on = ['Province_State', 'Country_Region'])
    data_deaths_US = data_deaths_US.merge(lat_long_avg, how = 'left',
                                               left_on = ['Province_State', 'Country_Region'],
                                               right_on = ['Province_State', 'Country_Region'])
    # raname colone
    data_confirmed_US = data_confirmed_US.rename(columns = {'Province_State':'Province/State', 'Country_Region': 'Country/Region', 'Long_':'Long'})
    data_deaths_US = data_deaths_US.rename(columns = {'Province_State':'Province/State', 'Country_Region': 'Country/Region', 'Long_':'Long'})

    # column order
    data_confirmed_US = data_confirmed_US[data_confirmed.columns]
    data_deaths_US = data_deaths_US[data_confirmed.columns]

    # suprimer ligne US de global conformed et deaths, en veut ajouter les donnée de US par sate et non pas la somme tatal 
    data_confirmed = data_confirmed[data_confirmed['Country/Region'] != 'US']
    data_deaths = data_deaths[data_deaths['Country/Region'] != 'US']

    #ajouter les donnée US au donnée global
    data_confirmed = pd.concat([data_confirmed, data_confirmed_US], ignore_index= True)
    data_deaths = pd.concat([data_deaths, data_deaths_US], ignore_index= True)
    
    # un-Pivotin gthe data
    # colone valeur contenant les valeur des cas de chaque jours(cumulé)

    data_confirmed2 = pd.melt(data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
    data_deaths2 =  pd.melt(data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
    data_recovered2 =  pd.melt(data_recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
    
    # Convertir la nouvelle colone Date to date type
    data_confirmed2.Date = pd.to_datetime(data_confirmed2.Date)
    data_deaths2.Date = pd.to_datetime(data_deaths2.Date)
    data_recovered2.Date = pd.to_datetime(data_recovered2.Date)
    
    # Renomer les colone value 
    data_confirmed2.columns = data_confirmed2.columns.str.replace("value", "Confirmed")
    data_deaths2.columns = data_deaths2.columns.str.replace('value', "Deaths")
    data_recovered2.columns = data_recovered2.columns.str.replace("value", 'Recovered')
    
    # gérer les valeurs manquantes de la variable "Province/State" : fill them with Coubtry/regiin values
    data_confirmed2['Province/State'].fillna(np.nan, inplace = True)
    data_deaths2['Province/State'].fillna(np.nan, inplace = True)
    data_recovered2['Province/State'].fillna(np.nan, inplace = True)

    data_confirmed2['Province/State'].fillna(data_confirmed2['Country/Region'], inplace = True)
    data_deaths2['Province/State'].fillna(data_deaths2['Country/Region'], inplace =True)
    data_recovered2['Province/State'].fillna(data_recovered2['Country/Region'], inplace = True)
    
    # confirmer + deaths + recovery
    full_data = data_confirmed2.merge(data_deaths2[['Province/State', 'Country/Region', 'Date', 'Deaths']],
                                     how= 'left',
                                     left_on = ['Province/State', 'Country/Region', 'Date'],
                                     right_on= ['Province/State', 'Country/Region', 'Date']
                                     )

    full_data = full_data.merge(data_recovered2[['Province/State', 'Country/Region', 'Date', 'Recovered']],
                                how= 'left',
                                left_on = ['Province/State', 'Country/Region', 'Date'],
                                right_on= ['Province/State', 'Country/Region', 'Date'])
    
    full_data["Recovered"].fillna(np.nan, inplace = True)
    full_data["Recovered"].fillna(0, inplace = True)
    full_data['Recovered'] = full_data['Recovered'].astype('int64')
    
    # Ajouter une nouvelle colone contenant l'année et le mois : sera utile dans la parite viz
    #full_data['Month-Year'] = full_data['Date'].dt.strftime('%b-%Y')
    
    # créer des colone contenant daily confirmed, daily death and daily recovered cases 
    # new df
    full_data2 = full_data.copy()

    #new colone date - 1 
    full_data2['Date-1'] = full_data2['Date'] + pd.Timedelta(days = 1)
    full_data2.rename(columns = {'Confirmed': 'Confirmed - 1', 'Deaths':'Deaths - 1', 'Recovered': 'Recovered - 1',
                                'Date': 'Date Minus 1'}, inplace = True)
    
    #join mydata with the df created
    full_data3 = full_data.merge(full_data2[['Province/State', 'Country/Region', 'Confirmed - 1', 'Deaths - 1', 'Recovered - 1',
                                             'Date-1', 'Date Minus 1']], 
                                 how= "left",
                                 left_on= ['Province/State', 'Country/Region', 'Date'],
                                 right_on= ['Province/State', 'Country/Region', 'Date-1'])
    
    # calcule des cas par jour
    full_data3['Confirmed Daily'] = full_data3['Confirmed'] - full_data3['Confirmed - 1']
    full_data3['Deaths Daily'] = full_data3["Deaths"] - full_data3['Deaths - 1']
    full_data3['Recovered Daily'] = full_data3["Recovered"] - full_data3['Recovered - 1']
    
    # ajouter manuellment les valeurs pour le premier jours de notre jeux de donnée
    full_data3['Confirmed Daily'].loc[full_data3['Date'] == '2020-01-22'] = full_data3['Confirmed']
    full_data3['Deaths Daily'].loc[full_data3['Date'] == '2020-01-22'] = full_data3['Deaths']
    full_data3['Recovered Daily'].loc[full_data3['Date'] == '2020-01-22'] = full_data3['Recovered']

    full_data3 = full_data3.drop(['Confirmed - 1', 'Deaths - 1',
           'Recovered - 1', 'Date-1', 'Date Minus 1'], axis= 1 )
    
    # remplacer les val nan dans recovery avec 0
    full_data3["Recovered Daily"].fillna(np.nan, inplace = True)
    full_data3["Recovered Daily"].fillna(0, inplace = True)
    
    # change variable de type float au type int 
    full_data3['Confirmed Daily'] = full_data3['Confirmed Daily'].astype('int64')
    full_data3['Deaths Daily'] = full_data3['Deaths Daily'].astype('int64')
    full_data3['Recovered Daily'] = full_data3['Recovered Daily'].astype('int64')
    full_data3['Recovered'] = full_data3['Recovered'].astype('int64')
    
    # Moratlaity rate & recovery rate
    for ind, row in full_data3.iterrows():
        full_data3.loc[ind, 'Recovery Rate'] = round(row["Recovered Daily"] / row['Confirmed Daily'] if row['Confirmed Daily'] != 0 else 0, 3)
        full_data3.loc[ind, 'Mortality Rate'] = round(row["Deaths Daily"] / row['Confirmed Daily'] if row['Confirmed Daily'] != 0 else 0, 3)


    # Exporting teh data to csv
    #full_data3.to_csv('Corono.csv', sep ='\t', index = False)
    
    # export data to sql
    full_data3.to_sql('Daily_Covid_Data', schema='dbo', con = engine, if_exists = 'replace')
    
    




NameError: name 'podbc' is not defined

In [ ]:
schedule.every().day.at('00:01').do(data_ETL)

while True:
    schedule.run_pending()
    time.sleep(999)